In [68]:
from csv import reader
# Apple store data
opened1 = open('AppleStore.csv')
read1 = reader(opened1)
lis1 = list(read1)
apple = lis1[1:]

# Google playstore data
opened2 = open('googleplaystore.csv')
read2 = reader(opened2)
lis2 = list(read2)
android = lis2[1:]

# Function for exploring data
def explore_data(dataset,start,end,rows_and_columns = False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
    
    if rows_and_columns:
        print('Number of rows:',len(dataset))
        print('Number of columns',len(dataset[0]))
        
        

In [69]:
# Explore Apple data
print('IOS header\n',lis1[0])
print('\n')
explore_data(apple,1,3,True)

IOS header
 ['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns 16


We have 7197 iOS apps in this data set and 16 columns. The columns that seem interesting are: 'track_name', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', and 'prime_genre'. Not all column names are self-explanatory in this case, but details about each column can be found in the data set [documentation](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home). 

In [70]:
# Explore Android data
print('Android header\n',lis2[0])
print('\n')
explore_data(android,1,3,True)

Android header
 ['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns 13


We  can see that the Google Play data set has 10841 apps and 13 columns. On looking at the column names,the columns that might be useful for the purpose of our analysis are 'App', 'Category', 'Reviews', 'Installs', 'Type', 'Price', and 'Genres'.


# DATA CLEANING

## 1. Deleting Wrong Data 
From Google's [discussion section] (https://www.kaggle.com/lava18/google-play-store-apps/discussion) , in one of the discussion an error is indicated for row number 10472. Lets print this row and investigate if there is any error related to the data column we require 

In [71]:
print(android[10472])

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


As mentioned in the discussion and on investigation we found that the data of this row is missing category column and hence should be deleted from the dataset.

In [72]:
del android[10472]

In [73]:
print(android[10472])

['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


## 2. Removing Duplicate Data 

After analysing Google playstore we noticed that there are apps with entries more than once and should be avoided for correct data analytics. For example Instagram occurs multiple time in the dataset.

In [74]:
for app in android:
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


In [75]:
duplicate_apps = [ ]
unique_apps = [ ]
for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
print('Number of duplicate apps', len(duplicate_apps))
print('Examples of Duplicated apps are:',duplicate_apps[:7])

Number of duplicate apps 1181
Examples of Duplicated apps are: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits']


As we can see there are 1181 apps duplicated. In our analysis we dont want a particular app to be counted more than once. Hence we'll be keeping only one entry per app and removing rest of the entries of a particular app by using somelogic.
As we can see from the duplicates of `Instagram app` the fourth column differed for every entry, which corresponds to number of reviews. The different numbers show that the data was collected at different times.he higher the number of reviews, the more recent the data should be. Rather than removing duplicates randomly, we'll only keep the row with the highest number of reviews and remove the other entries for any given app.

In [76]:
print('Expected length', len(android)-1181)

Expected length 9659


Above we calculated number of valid entries by removing all unwanted duplicates from the dataset. After we remove the duplicates, we should be left with 9,659 rows.

To remove the duplicates, we will:

1. Create a dictionary, where each dictionary key is a unique app name and the corresponding dictionary value is the highest number of reviews of that app
2. Use the information stored in the dictionary to create a new data set, which will have only one entry per app (and for each app, we'll only select the entry with the highest number of reviews)

In [77]:
reviews_max = { }
for row in android:
    name = row[0]
    n_reviews = float(row[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
print(len(reviews_max))

9659


Now, let's use the reviews_max dictionary to remove the duplicates. For the duplicate cases, we'll only keep the entries with the highest number of reviews. In the code cell below:

We start by initializing two empty lists, android_clean and already_added.
1. We loop through the android data set, and for every iteration.
2. We isolate the name of the app and the number of reviews.
3. We add the current row (app) to the android_clean list, and the app name (name) to the already_cleaned list if.
* The number of reviews of the current app matches the number of reviews of that app as described in the reviews_max dictionary; and
* The name of the app is not already in the already_added list. We need to add this supplementary condition to account for those cases where the highest number of reviews of a duplicate app is the same for more than one entry (for example, the Box app has three entries, and the number of reviews is the same). If we just check for reviews_max[name] == n_reviews, we'll still end up with duplicate entries for some apps.

In [78]:
android_clean = [ ] 
already_added = [ ]
for row in android:
    name = row[0]
    n_reviews = float(row[3])
    if (n_reviews == reviews_max[name] ) and (name not in already_added):
        android_clean.append(row)
        already_added.append(name)
        

Now lets use the android_clean data set and explore its data:-

In [79]:
explore_data(android_clean,1,3,True)

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns 13


## 3. Removing Non-English Apps

Now after removing incorrect data and duplicate data from dataset the next step in data cleaning of this dataset is elimination of Non-English apps. The language we use for the apps we develop at our company is English, and we'd like to analyze only the apps that are directed toward an English-speaking audience. However, if we explore the data long enough, we'll find that both data sets have apps whose name suggests that they are not directed toward an English-speaking audience.

In [80]:
print(apple[813][1])
print('\n')
print(android_clean[4412][0])

爱奇艺PPS -《欢乐颂2》电视剧热播


中国語 AQリスニング


We're not interested in keeping these kind of apps, so we'll remove them. One way to go about this is to remove each app whose name contains a symbol that is not commonly used in English text — English text usually includes letters from the English alphabet, numbers composed of digits from 0 to 9, punctuation marks (., !, ?, ;, etc.), and other symbols (+, *, /, etc.).

All these characters that are specific to English texts are encoded using the ASCII standard. Each ASCII character has a corresponding number between 0 and 127 associated with it, and we can take advantage of that to build a function that checks an app name and tells us whether it contains non-ASCII characters.

We built this function below, and we use the built-in **ord()** function to find out the corresponding encoding number of each character.

In [81]:
def string_character(string):
    for c in string:
        if (ord(c) > 127):
            return False
    return True

In [82]:
print(string_character('Instagram'))
print(string_character('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(string_character('Docs To Go™ Free Office Suite'))
print(string_character('Instachat 😜'))

True
False
False
False


This function works well but when an emoji is used with english language, it returns False as shown in the last print statement.To minimize this error we will allow maximum of 3 character with ASCII value greater than 127. Below is the following function 

In [83]:
def string_character(string):
    a = 0
    for c in string:
        if ord(c) > 127:
            a += 1
    if a > 3:
        return False
    else: 
        return True

In [84]:
print(string_character('Instagram'))
print(string_character('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(string_character('Docs To Go™ Free Office Suite'))
print(string_character('Instachat 😜'))

True
False
True
True


Now we'll remove non-english apps from our most refined android and IOS dataset by using the string_character function.

In [85]:
# For IOS
ios_refined = [ ]
for row in apple:
    name = row[1]
    if string_character(name):
        ios_refined.append(row)
explore_data(ios_refined,1,3,True)
      

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 6183
Number of columns 16


In [86]:
# For Android
android_refined = [ ]
for row in android_clean:
    name = row[0]
    if string_character(name):
        android_refined.append(row)
explore_data(android_refined,1,3,True)

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns 13


As we are intrested in free apps and the dataset above contains both free and non-free apps and we'll need to isolate only the free apps for our analysis. Isolating the free apps will be our last step in the data cleaning process. 

In [87]:
# For Apple
apple_cleaned = [ ]
for row in ios_refined:
    price = row[4]
    if price == '0.0':
        apple_cleaned.append(row)
print(len(apple_cleaned))


3222


In [88]:
# For Android
android_cleaned= [ ]
for row in android_refined:
    if row[7] == '0':
        android_cleaned.append(row)
print(len(android_cleaned))

8864


We are left with 3222 ios apps and 8864 android apps, which is enough for our analysis.

## Most common Apps by Genre
Our aim is to determine apps that are more likely to attract users. To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we then develop it further.
3. If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store.

Let's begin the analysis by getting a sense of what are the most common genres for each market.

We'll build two functions we can use to analyze the frequency tables:

1. One function to generate frequency tables that show percentages
2. Another function we can use to display the percentages in a descending order

In [89]:
print(android_cleaned[1:3])

[['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'], ['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']]


In [90]:
def freq_table(dataset,index):
    frequency = { }
    f_total = 0
    for row in dataset:
        column = row[index]
        f_total += 1
        if column in frequency:
            frequency[column] += 1
        else:
            frequency[column] = 1
    
    
    frequency_perc = { }
    for key in frequency:
        perc = (frequency[key] / f_total)*100
        frequency_perc[key] = perc
    return frequency_perc


def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

Now we will examine the above two functions on prime_genre of apple to obtain publicily fvourite genre (frequency of genre in descending order).

In [91]:
display_table(apple_cleaned,-5)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


From the above obtained result, we obatin that the most common genre is `Games`,that is genre with maximum frequency,following this is `Entertainment`. It can be seen that most of the apps on apple store are games and that is approximately 50% more than the second place entertainment. Where as apps for medical purpose is less than one percent of the number of apps in our dataset, which may mean apps related to medical and health are few in numbers and trusted brands. The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users — the demand might not be the same as the offer.

Now we use the functions on android data for columns `category` and `genres`

In [92]:
# for column 'Category'
display_table(android_cleaned,1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

In [93]:
# For column 'Genres'
display_table(android_cleaned,-4)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

The difference between the Genres and the Category columns is not crystal clear, but one thing we can notice is that the Genres column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the Category column moving forward.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.


## Most Popular Apps on App Store

One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the Installs column, but this information is missing for the App Store data set. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the rating_count_tot app.



Let's take a look at the column names of IOS data to obtain the column number of *rating_count_tot app*

In [94]:
print(lis1[0])

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


In [95]:

genre_ios = freq_table(apple_cleaned,-5)
rating_dict = { }

for genre in genre_ios:
    total = 0
    len_genre = 0
    for row in apple_cleaned:
        genre_app = row[-5]
        if genre_app == genre:
            n_rating = float(row[5])
            total += n_rating
            len_genre += 1
    avg_rating = total / len_genre
    print(genre ,':', avg_rating)
    rating_dict[genre] = avg_rating
print(rating_dict)    

Lifestyle : 16485.764705882353
Shopping : 26919.690476190477
News : 21248.023255813954
Medical : 612.0
Sports : 23008.898550724636
Productivity : 21028.410714285714
Social Networking : 71548.34905660378
Travel : 28243.8
Photo & Video : 28441.54375
Weather : 52279.892857142855
Food & Drink : 33333.92307692308
Reference : 74942.11111111111
Finance : 31467.944444444445
Music : 57326.530303030304
Navigation : 86090.33333333333
Games : 22788.6696905016
Health & Fitness : 23298.015384615384
Education : 7003.983050847458
Entertainment : 14029.830708661417
Business : 7491.117647058823
Utilities : 18684.456790123455
Catalogs : 4004.0
Book : 39758.5
{'Lifestyle': 16485.764705882353, 'Shopping': 26919.690476190477, 'News': 21248.023255813954, 'Medical': 612.0, 'Sports': 23008.898550724636, 'Productivity': 21028.410714285714, 'Social Networking': 71548.34905660378, 'Travel': 28243.8, 'Photo & Video': 28441.54375, 'Weather': 52279.892857142855, 'Food & Drink': 33333.92307692308, 'Reference': 74942.

Now we use *sorted* function to arrange the average rating of all IOS apps in descending order of average ratings 

In [96]:
def sortdict (dic):
    lis_tup = [ ]
    for key in dic:
        tuple_dic = (dic[key],key)
        lis_tup.append(tuple_dic)
    sorted_dic = sorted(lis_tup,reverse = True)
    for value in sorted_dic:
        print(value[1],':', value[0])
    

In [97]:
sortdict(rating_dict)

Navigation : 86090.33333333333
Reference : 74942.11111111111
Social Networking : 71548.34905660378
Music : 57326.530303030304
Weather : 52279.892857142855
Book : 39758.5
Food & Drink : 33333.92307692308
Finance : 31467.944444444445
Photo & Video : 28441.54375
Travel : 28243.8
Shopping : 26919.690476190477
Health & Fitness : 23298.015384615384
Sports : 23008.898550724636
Games : 22788.6696905016
News : 21248.023255813954
Productivity : 21028.410714285714
Utilities : 18684.456790123455
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Business : 7491.117647058823
Education : 7003.983050847458
Catalogs : 4004.0
Medical : 612.0


On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together.

In [98]:

for row in apple_cleaned:
    column = row[-5]
    if column == 'Navigation':
        print(row[1],':',row[5])

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


Same goes for social networking where the average rating is majorly influenced by giant apps like Facebook, Pinterest, etc. In music genre also the average rating is influenced by apps like Spotify, Pandora,etc.

In [99]:
for row in apple_cleaned:
    column = row[-5]
    if column == 'Social Networking':
        print(row[1],':',row[5])
        
print('\n')

for row in apple_cleaned:
    column = row[-5]
    if column == 'Music':
        print(row[1],':',row[5])


Facebook : 2974676
Pinterest : 1061624
Skype for iPhone : 373519
Messenger : 351466
Tumblr : 334293
WhatsApp Messenger : 287589
Kik : 260965
ooVoo – Free Video Call, Text and Voice : 177501
TextNow - Unlimited Text + Calls : 164963
Viber Messenger – Text & Call : 164249
Followers - Social Analytics For Instagram : 112778
MeetMe - Chat and Meet New People : 97072
We Heart It - Fashion, wallpapers, quotes, tattoos : 90414
InsTrack for Instagram - Analytics Plus More : 85535
Tango - Free Video Call, Voice and Chat : 75412
LinkedIn : 71856
Match™ - #1 Dating App. : 60659
Skype for iPad : 60163
POF - Best Dating App for Conversations : 52642
Timehop : 49510
Find My Family, Friends & iPhone - Life360 Locator : 43877
Whisper - Share, Express, Meet : 39819
Hangouts : 36404
LINE PLAY - Your Avatar World : 34677
WeChat : 34584
Badoo - Meet New People, Chat, Socialize. : 34428
Followers + for Instagram - Follower Analytics : 28633
GroupMe : 28260
Marco Polo Video Walkie Talkie : 27662
Miitomo : 2

Our aim is to find popular genres. But genres having giant apps like Navigaton, Social Networking, Music might look more more popular than they really are. One of the surprising entry in this list of genres is the genre *Reference*.

In [100]:
for row in apple_cleaned:
    column = row[-5]
    if column == 'Reference':
        print(row[1],':',row[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


The reference genre is skewed by apps like Bible and Dictionary.com. But this apps are not part of any giant company like Facebook, etc. So it will be better for the business as the competition will be limited and profitbale. We can use this genre to make apps like digital book, puzzle on different books, daily philosophical quotes,etc. and as the trend for IOS  market is for fun apps these ideas may work well.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:

Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.

Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.

Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

## Most Popular Apps by Genre on Google Play

We have data about the number of installs for the Google Play market, so we should be able to get a clearer picture about genre popularity. However, the install numbers don't seem precise enough — we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.):



In [101]:
display_table(android_cleaned,5)

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


One problem with this data is that it's not precise. For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to find out which app genres attract the most users, and we don't need perfect precision with respect to the number of users.

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on. To perform computations, however, we'll need to convert each install number from string to float. This means we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error. 

We will do this conversion with the help of a for loop and use str.replace(old, new) method . First lets go through the column name for android apps data. 

In [102]:
print(lis2[0])

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [103]:
category_freq = freq_table(android_cleaned,1)
dict_install = { }
for category in category_freq:
    total= 0
    len_category = 0
    for row in android_cleaned:
        column = row[1]
        if column == category:
            total_install = row[5]
            total_install = total_install.replace('+','')
            total_install = total_install.replace(',','')
            total += float(total_install)
            len_category += 1
    avg_install = total/len_category
    print(category,':',avg_install)
    dict_install[category] = avg_install


EDUCATION : 1833495.145631068
MAPS_AND_NAVIGATION : 4056941.7741935486
ART_AND_DESIGN : 1986335.0877192982
SHOPPING : 7036877.311557789
ENTERTAINMENT : 11640705.88235294
VIDEO_PLAYERS : 24727872.452830188
BEAUTY : 513151.88679245283
PHOTOGRAPHY : 17840110.40229885
LIBRARIES_AND_DEMO : 638503.734939759
BOOKS_AND_REFERENCE : 8767811.894736841
PARENTING : 542603.6206896552
HOUSE_AND_HOME : 1331540.5616438356
FOOD_AND_DRINK : 1924897.7363636363
MEDICAL : 120550.61980830671
GAME : 15588015.603248259
COMICS : 817657.2727272727
AUTO_AND_VEHICLES : 647317.8170731707
PRODUCTIVITY : 16787331.344927534
COMMUNICATION : 38456119.167247385
BUSINESS : 1712290.1474201474
DATING : 854028.8303030303
LIFESTYLE : 1437816.2687861272
TRAVEL_AND_LOCAL : 13984077.710144928
NEWS_AND_MAGAZINES : 9549178.467741935
HEALTH_AND_FITNESS : 4188821.9853479853
TOOLS : 10801391.298666667
FAMILY : 3695641.8198090694
EVENTS : 253542.22222222222
PERSONALIZATION : 5201482.6122448975
SOCIAL : 23253652.127118643
WEATHER : 507

We've stored the results in a dictionary named dict_install, so that we can sort them later using *sorted* function for easy analysis. 

In [104]:
print(dict_install)

{'MAPS_AND_NAVIGATION': 4056941.7741935486, 'PERSONALIZATION': 5201482.6122448975, 'ART_AND_DESIGN': 1986335.0877192982, 'SPORTS': 3638640.1428571427, 'SHOPPING': 7036877.311557789, 'ENTERTAINMENT': 11640705.88235294, 'VIDEO_PLAYERS': 24727872.452830188, 'PHOTOGRAPHY': 17840110.40229885, 'LIBRARIES_AND_DEMO': 638503.734939759, 'BOOKS_AND_REFERENCE': 8767811.894736841, 'PARENTING': 542603.6206896552, 'HOUSE_AND_HOME': 1331540.5616438356, 'FOOD_AND_DRINK': 1924897.7363636363, 'MEDICAL': 120550.61980830671, 'GAME': 15588015.603248259, 'COMICS': 817657.2727272727, 'AUTO_AND_VEHICLES': 647317.8170731707, 'PRODUCTIVITY': 16787331.344927534, 'FINANCE': 1387692.475609756, 'BUSINESS': 1712290.1474201474, 'DATING': 854028.8303030303, 'LIFESTYLE': 1437816.2687861272, 'TRAVEL_AND_LOCAL': 13984077.710144928, 'NEWS_AND_MAGAZINES': 9549178.467741935, 'HEALTH_AND_FITNESS': 4188821.9853479853, 'TOOLS': 10801391.298666667, 'FAMILY': 3695641.8198090694, 'EDUCATION': 1833495.145631068, 'COMMUNICATION': 38

In [105]:
def desc_install(avg_install_dict):
    lis_tuple_install = [ ]
    for key in avg_install_dict:
        dict_to_tuple = (avg_install_dict[key],key)
        lis_tuple_install.append(dict_to_tuple)
    sorted_avg_install = sorted(lis_tuple_install,reverse = True)
    for r in sorted_avg_install:
        print(r[1],':',r[0])

The sorted list is:-

In [106]:
desc_install(dict_install)

COMMUNICATION : 38456119.167247385
VIDEO_PLAYERS : 24727872.452830188
SOCIAL : 23253652.127118643
PHOTOGRAPHY : 17840110.40229885
PRODUCTIVITY : 16787331.344927534
GAME : 15588015.603248259
TRAVEL_AND_LOCAL : 13984077.710144928
ENTERTAINMENT : 11640705.88235294
TOOLS : 10801391.298666667
NEWS_AND_MAGAZINES : 9549178.467741935
BOOKS_AND_REFERENCE : 8767811.894736841
SHOPPING : 7036877.311557789
PERSONALIZATION : 5201482.6122448975
WEATHER : 5074486.197183099
HEALTH_AND_FITNESS : 4188821.9853479853
MAPS_AND_NAVIGATION : 4056941.7741935486
FAMILY : 3695641.8198090694
SPORTS : 3638640.1428571427
ART_AND_DESIGN : 1986335.0877192982
FOOD_AND_DRINK : 1924897.7363636363
EDUCATION : 1833495.145631068
BUSINESS : 1712290.1474201474
LIFESTYLE : 1437816.2687861272
FINANCE : 1387692.475609756
HOUSE_AND_HOME : 1331540.5616438356
DATING : 854028.8303030303
COMICS : 817657.2727272727
AUTO_AND_VEHICLES : 647317.8170731707
LIBRARIES_AND_DEMO : 638503.734939759
PARENTING : 542603.6206896552
BEAUTY : 51315

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [112]:
for row in android_cleaned:
    if (row[1] == 'COMMUNICATION') & (row[5] == '1,000,000,000+' or row[5] == '500,000,000+' or row[5] == '100,000,000+'):
         print(row[0],':',row[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

If we filter the data before calculating average installs by removing skewed apps i.e. removing  communication apps having installs greater than 100 million.The average we'll get will obviously lesser than the previous average

In [125]:
filter_under_100m = [ ]
for row in android_cleaned:
    column = row[5]
    column = column.replace('+','')
    column = column.replace(',','')
    if (row[1] == 'COMMUNICATION') and (float(column) < 100000000):
        filter_under_100m.append(float(column))
print(sum(filter_under_100m)/len(filter_under_100m))
        

3603485.3884615386


We see the same pattern for the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Again, the main concern is that these app genres might seem more popular than they really are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against.

The game genre seems pretty popular, but previously we found out this part of the market seems a bit saturated, so we'd like to come up with a different app recommendation if possible.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

Let's take a look at some of the apps from this genre and their number of installs:

In [126]:
for row in android_cleaned:
    if row[1] == 'BOOKS_AND_REFERENCE':
        print(row[0],':',row[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

The book and reference genre includes a variety of apps: software for processing and reading ebooks, various collections of libraries, dictionaries, tutorials on programming or languages, etc. It seems there's still a small number of extremely popular apps that skew the average:

In [137]:
for row in android_cleaned:
    if row[1] == 'BOOKS_AND_REFERENCE' and (row[5] == '1,000,000,000+'
                                            or row[5] == '500,000,000+'
                                            or row[5] == '100,000,000+'):
        print(row[0], ':', row[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


However, it looks like there are only a few very popular apps, so this market still shows potential. Let's try to get some app ideas based on the kind of apps that are somewhere in the middle in terms of popularity (between 1,000,000 and 100,000,000 downloads):

In [139]:
for row in android_cleaned:
    if (row[1] == 'BOOKS_AND_REFERENCE') and (row[5] == '1,000,000+' or row[5] == '5,000,000+'or row[5] == '10,000,000+' or row[5] == '50,000,000+'):
        print(row[0],':',row[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

We notice that there are many apps built on popular books like holy books, dictonary, educational books,etc. based on this we can create apps related to books but that apps must be different from the above list. We also know that users love fun app or games app sowe can integrate game and books in an app. The market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

## Conclusion

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.
We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, vaocabulary quiz, engineering book formula or concepts libraries, etc.